## **Import Library**

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0563f6c22fd82364b156a1d534d9f8860feeefa3119dd4df238d9a34b9e641d3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_scheduler
from rouge_score import rouge_scorer

In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## **Load Dataset**

In [ ]:
dataset = load_dataset("allenai/scitldr", "AIC")  # Scientific paper dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

scitldr.py:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/7.23M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1992 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/618 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/619 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'source_labels', 'rouge_scores', 'paper_id', 'ic', 'target'],
        num_rows: 1992
    })
    test: Dataset({
        features: ['source', 'source_labels', 'rouge_scores', 'paper_id', 'ic', 'target'],
        num_rows: 618
    })
    validation: Dataset({
        features: ['source', 'source_labels', 'rouge_scores', 'paper_id', 'ic', 'target'],
        num_rows: 619
    })
})

In [ ]:
train_data = dataset['train']
test_data = dataset['test']
val_data = dataset['validation']

In [ ]:
# from torch.utils.data import Dataset, DataLoader

# class SCITldrDataset(Dataset):
#     def __init__(self, dataset, tokenizer, max_input_length=512, max_target_length=100):
#         self.dataset = dataset  # Save the dataset object directly
#         self.tokenizer = tokenizer
#         self.max_input_length = max_input_length
#         self.max_target_length = max_target_length

#     def __len__(self):
#         return len(self.dataset)  # Use the length of the dataset

#     def __getitem__(self, idx):
#         item = self.dataset[idx]  # Access an individual data entry
#         # Join the list of strings in 'source' into a single string
#         input_text = "summarize: " + " ".join(item["source"])
#         target_text = item["target"][0]

#         # Tokenize input and output
#         inputs = self.tokenizer(
#             input_text,
#             max_length=self.max_input_length,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )
#         targets = self.tokenizer(
#             target_text,
#             max_length=self.max_target_length,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": inputs["input_ids"].squeeze(),
#             "attention_mask": inputs["attention_mask"].squeeze(),
#             "labels": targets["input_ids"].squeeze(),
#         }

    # Unused

In [ ]:
class SCITldrDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset  # Save the dataset object directly
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)  # Use the length of the dataset

    def __getitem__(self, idx):
        item = self.dataset[idx]  # Access an individual data entry
        # Join the list of strings in 'source' into a single string
        input_text = "summarize: " + " ".join(item["source"])
        target_text = item["target"][0]

        # Tokenize input and output
        inputs = self.tokenizer(
            input_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        targets = self.tokenizer(
            target_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze(),
        }

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Prepare datasets
train_dataset = SCITldrDataset(train_data, tokenizer)
test_dataset = SCITldrDataset(test_data, tokenizer)
val_dataset = SCITldrDataset(val_data, tokenizer)

In [ ]:
# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

## **Training Eval**

In [ ]:
# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 10  # Maximum number of epochs
patience = 3  # Stop if no improvement in 'patience' epochs
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"Epoch {epoch + 1}/{epochs} starts...")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print loss every 10 steps
        if (step + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, Step {step + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    # Print average loss for the epoch
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} completed. Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}")

    # Check for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pt")  # Save the best model
        print("Best model saved.")
    else:
        patience_counter += 1
        print(f"No improvement in validation loss. Patience counter: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10 starts...
Epoch 1, Step 10/249, Loss: 2.3655
Epoch 1, Step 20/249, Loss: 1.9945
Epoch 1, Step 30/249, Loss: 1.3635
Epoch 1, Step 40/249, Loss: 1.7604
Epoch 1, Step 50/249, Loss: 1.4738
Epoch 1, Step 60/249, Loss: 1.6566
Epoch 1, Step 70/249, Loss: 1.8524
Epoch 1, Step 80/249, Loss: 1.6616
Epoch 1, Step 90/249, Loss: 1.7265
Epoch 1, Step 100/249, Loss: 2.1374
Epoch 1, Step 110/249, Loss: 1.8448
Epoch 1, Step 120/249, Loss: 1.5521
Epoch 1, Step 130/249, Loss: 1.2318
Epoch 1, Step 140/249, Loss: 1.8143
Epoch 1, Step 150/249, Loss: 1.5127
Epoch 1, Step 160/249, Loss: 1.1450
Epoch 1, Step 170/249, Loss: 1.5370
Epoch 1, Step 180/249, Loss: 1.0803
Epoch 1, Step 190/249, Loss: 2.0053
Epoch 1, Step 200/249, Loss: 1.4671
Epoch 1, Step 210/249, Loss: 1.9224
Epoch 1, Step 220/249, Loss: 1.5151
Epoch 1, Step 230/249, Loss: 1.2950
Epoch 1, Step 240/249, Loss: 1.4456
Epoch 1/10 completed. Training Loss: 1.6765
Validation Loss: 1.4535
Best model saved.
Epoch 2/10 starts...
Epoch 2, Step 10/

In [ ]:
# Load the best model
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

predictions = []
references = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=128)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_refs = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

        predictions.extend(decoded_preds)
        references.extend(decoded_refs)

<ipython-input-19-f9035c2b42aa>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))
Setting `pad_token_id` to `eos_token_id`:

In [ ]:
# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

rouge1, rouge2, rougeL = 0, 0, 0
for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge1 += scores["rouge1"].fmeasure
    rouge2 += scores["rouge2"].fmeasure
    rougeL += scores["rougeL"].fmeasure

n = len(predictions)
print(f"ROUGE-1: {rouge1 / n:.4f}")
print(f"ROUGE-2: {rouge2 / n:.4f}")
print(f"ROUGE-L: {rougeL / n:.4f}")

ROUGE-1: 0.2207
ROUGE-2: 0.0945
ROUGE-L: 0.1587
